In [1]:
!pip install pulp
!sudo apt install libglpk-dev python3.8-dev libgmp3-dev
!pip install glpk
!pip install rouge-score

import pulp
import nltk
import numpy as np
from rouge_score import rouge_scorer
import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 40.6MB 79kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas1 libklu1 libldl2 libmetis5
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  librbio2 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
  python3.8 python3.8-minimal
Suggested packages:
  libiodbc2-dev default-libmysqlclient-dev gmp-doc libgmp10-doc libmpfr-dev
  python3.8-venv python3.8-doc binfmt-support
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk-dev libglpk40 libgmp-dev libgmp3-dev libgmpxx4ldbl libgraphblas1
  libklu1 libldl2 libmetis5 libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib librbio2 libspqr2 libsuitespa

True

# ILP Summary model

In [3]:
import re
import os
import codecs
from collections import Counter

class State:
    """ State class
    Internal class used as a structure to keep track of the search state in 
    the tabu_search method.
    Args:
        subset (set): a subset of sentences
        concepts (Counter): a set of concepts for the subset
        length (int): the length in words
        score (int): the score for the subset
    """
    def __init__(self):
        self.subset = set()
        self.concepts = Counter()
        self.length = 0
        self.score = 0

class Sentence:
    """The sentence data structure.
    Args: 
        tokens (list of str): the list of word tokens.
        doc_id (str): the identifier of the document from which the sentence
          comes from.
        position (int): the position of the sentence in the source document.
    """
    def __init__(self, tokens, doc_id, position):

        self.tokens = tokens
        """ tokens as a list. """

        self.doc_id = doc_id
        """ document identifier of the sentence. """

        self.position = position
        """ position of the sentence within the document. """

        self.concepts = []
        """ concepts of the sentence. """

        self.untokenized_form = ''
        """ untokenized form of the sentence. """

        self.length = 0
        """ length of the untokenized sentence. """

class LoadFile(object):
    """Objects which inherit from this class have read file functions.
    """

    def __init__(self, input_directory):
        """
        Args:
            input_file (str): the path of the input file.
            use_stems (bool): whether stems should be used instead of words,
              defaults to False.
        """
        self.input_directory = input_directory
        self.sentences = []

    def read_documents(self, file_extension="txt"):
        """Read the input files in the given directory.
        Load the input files and populate the sentence list. Input files are
        expected to be in one tokenized sentence per line format.
        Args:
            file_extension (str): the file extension for input documents,
              defaults to txt.
        """


        with codecs.open(self.input_directory,
                            'r',
                            'utf-8',errors='replace') as f:

            # load the sentences
            lines = f.readlines()

            # loop over sentences
            for i in range(len(lines)):

                # split the sentence into tokens
                tokens = lines[i].strip().split(' ')

                # add the sentence
                infile = self.input_directory.split('/')[-1]
                if len(tokens) > 0:
                    sentence = Sentence(tokens, infile, i)
                    untokenized_form = untokenize(tokens)
                    sentence.untokenized_form = untokenized_form
                    sentence.length = len(untokenized_form.split(' '))
                    self.sentences.append(sentence)

def untokenize(tokens):
    """Untokenizing a list of tokens. 
    Args:
        tokens (list of str): the list of tokens to untokenize.
    Returns:
        a string
    """
    text = u' '.join(tokens)
    text = re.sub(u"\s+", u" ", text.strip())
    text = re.sub(u" ('[a-z]) ", u"\g<1> ", text)
    text = re.sub(u" ([\.;,-]) ", u"\g<1> ", text)
    text = re.sub(u" ([\.;,-?!])$", u"\g<1>", text)
    text = re.sub(u" _ (.+) _ ", u" _\g<1>_ ", text)
    text = re.sub(u" \$ ([\d\.]+) ", u" $\g<1> ", text)
    text = text.replace(u" ' ", u"' ")
    text = re.sub(u"([\W\s])\( ", u"\g<1>(", text)
    text = re.sub(u" \)([\W\s])", u")\g<1>", text)
    text = text.replace(u"`` ", u"``")
    text = text.replace(u" ''", u"''")
    text = text.replace(u" n't", u"n't")
    text = re.sub(u'(^| )" ([^"]+) "( |$)', u'\g<1>"\g<2>"\g<3>', text)

    # times
    text = re.sub('(\d+) : (\d+ [ap]\.m\.)', '\g<1>:\g<2>', text)

    text = re.sub('^" ', '"', text)
    text = re.sub(' "$', '"', text)
    text = re.sub(u"\s+", u" ", text.strip())

    return text





""" Concept-based ILP summarization methods.
    authors: Florian Boudin (florian.boudin@univ-nantes.fr)
             Hugo Mougard (hugo.mougard@univ-nantes.fr)
    version: 0.2
    date: May 2015
"""








# from sume.base import Sentence, State, untokenize, LoadFile

from collections import defaultdict, deque

import os
import re
import codecs
import random
import sys

import nltk
import pulp


class ConceptBasedILPSummarizer(LoadFile):
    """Implementation of the concept-based ILP model for summarization.
    The original algorithm was published and described in:
      * Dan Gillick and Benoit Favre, A Scalable Global Model for Summarization,
        *Proceedings of the NAACL HLT Workshop on Integer Linear Programming for
        Natural Language Processing*, pages 10–18, 2009.
        
    """
    def __init__(self, input_directory):
        """
        Args:
            input_directory (str): the directory from which text documents to
              be summarized are loaded.
        """
        self.input_directory = input_directory
        self.sentences = []
        self.weights = {}
        self.c2s = defaultdict(set)
        self.concept_sets = defaultdict(frozenset)
        self.stoplist = nltk.corpus.stopwords.words('english')
        self.stemmer = nltk.stem.snowball.SnowballStemmer('english')
        self.word_frequencies = defaultdict(int)
        self.w2s = defaultdict(set)

    def extract_ngrams(self, n=2):
        """Extract the ngrams of words from the input sentences.
        Args:
            n (int): the number of words for ngrams, defaults to 2
        """
        for i, sentence in enumerate(self.sentences):

            # for each ngram of words
            for j in range(len(sentence.tokens)-(n-1)):

                # initialize ngram container
                ngram = []

                # for each token of the ngram
                for k in range(j, j+n):
                    ngram.append(sentence.tokens[k].lower())

                # do not consider ngrams containing punctuation marks
                marks = [t for t in ngram if not re.search('[a-zA-Z0-9]', t)]
                if len(marks) > 0:
                    continue

                # do not consider ngrams composed of only stopwords
                stops = [t for t in ngram if t in self.stoplist]
                if len(stops) == len(ngram):
                    continue

                # stem the ngram
                ngram = [self.stemmer.stem(t) for t in ngram]

                # add the ngram to the concepts
                self.sentences[i].concepts.append(' '.join(ngram))

    def compute_document_frequency(self):
        """Compute the document frequency of each concept.
        """
        for i in range(len(self.sentences)):

            # for each concept
            for concept in self.sentences[i].concepts:

                # add the document id to the concept weight container
                if concept not in self.weights:
                    self.weights[concept] = set([])
                self.weights[concept].add(self.sentences[i].doc_id)

        # loop over the concepts and compute the document frequency
        for concept in self.weights:
            self.weights[concept] = len(self.weights[concept])

    def compute_word_frequency(self):
        """Compute the frequency of each word in the set of documents. """

        for i, sentence in enumerate(self.sentences):
            for token in sentence.tokens:
                t = token.lower() 
                if not re.search('[a-zA-Z0-9]', t) or t in self.stoplist:
                    continue
                t = self.stemmer.stem(t)
                self.w2s[t].add(i)
                self.word_frequencies[t] += 1

    def prune_sentences(self,
                        mininum_sentence_length=5,
                        remove_citations=True,
                        remove_redundancy=True):
        """Prune the sentences.
        Remove the sentences that are shorter than a given length, redundant
        sentences and citations from entering the summary.
        Args:
            mininum_sentence_length (int): the minimum number of words for a
              sentence to enter the summary, defaults to 5
            remove_citations (bool): indicates that citations are pruned,
              defaults to True
            remove_redundancy (bool): indicates that redundant sentences are
              pruned, defaults to True
        """
        pruned_sentences = []

        # loop over the sentences
        for sentence in self.sentences:

            # prune short sentences
            if sentence.length < mininum_sentence_length:
                continue

            # prune citations
            first_token, last_token = sentence.tokens[0], sentence.tokens[-1]
            if remove_citations and \
               (first_token == u"``" or first_token == u'"') and \
               (last_token == u"''" or first_token == u'"'):
                continue

            # prune ___ said citations
            # if remove_citations and \
            #     (sentence.tokens[0]==u"``" or sentence.tokens[0]==u'"') and \
            #     re.search('(?i)(''|") \w{,30} (said|reported|told)\.$',
            #               sentence.untokenized_form):
            #     continue

            # prune identical and almost identical sentences
            if remove_redundancy:
                is_redundant = False
                for prev_sentence in pruned_sentences:
                    if sentence.tokens == prev_sentence.tokens:
                        is_redundant = True
                        break

                if is_redundant:
                    continue

            # otherwise add the sentence to the pruned sentence container
            pruned_sentences.append(sentence)

        self.sentences = pruned_sentences

    def prune_concepts(self, method="threshold", value=3):
        """Prune the concepts for efficient summarization.
        Args:
            method (str): the method for pruning concepts that can be whether
              by using a minimal value for concept scores (threshold) or using
              the top-N highest scoring concepts (top-n), defaults to
              threshold.
            value (int): the value used for pruning concepts, defaults to 3.
        """
        # 'threshold' pruning method
        if method == "threshold":

            # iterates over the concept weights
            concepts = self.weights.keys()
            for concept in concepts:
                if self.weights[concept] < value:
                    del self.weights[concept]

        # 'top-n' pruning method
        elif method == "top-n":

            # sort concepts by scores
            sorted_concepts = sorted(self.weights,
                                     key=lambda x: self.weights[x],
                                     reverse=True)

            # iterates over the concept weights
            concepts = self.weights.keys()
            for concept in concepts:
                if concept not in sorted_concepts[:value]:
                    del self.weights[concept]

        # iterates over the sentences
        for i in range(len(self.sentences)):

            # current sentence concepts
            concepts = self.sentences[i].concepts

            # prune concepts
            self.sentences[i].concepts = [c for c in concepts
                                          if c in self.weights]

    def compute_c2s(self):
        """Compute the inverted concept to sentences dictionary. """

        for i, sentence in enumerate(self.sentences):
            for concept in sentence.concepts:
                self.c2s[concept].add(i)

    def compute_concept_sets(self):
        """Compute the concept sets for each sentence."""

        for i, sentence in enumerate(self.sentences):
            for concept in sentence.concepts:
                self.concept_sets[i] |= {concept}

    def greedy_approximation(self, summary_size=100):
        """Greedy approximation of the ILP model.
        Args:
            summary_size (int): the maximum size in words of the summary,
              defaults to 100.
        Returns:
            (value, set) tuple (int, list): the value of the approximated
              objective function and the set of selected sentences as a tuple.
        """
        # initialize the inverted c2s dictionary if not already created
        if not self.c2s:
            self.compute_c2s()

        # initialize weights
        weights = {}

        # initialize the score of the best singleton
        best_singleton_score = 0

        # compute indices of our sentences
        sentences = range(len(self.sentences))

        # compute initial weights and fill the reverse index
        # while keeping track of the best singleton solution
        for i, sentence in enumerate(self.sentences):
            weights[i] = sum(self.weights[c] for c in set(sentence.concepts))
            if sentence.length <= summary_size\
               and weights[i] > best_singleton_score:
                best_singleton_score = weights[i]
                best_singleton = i

        # initialize the selected solution properties
        sel_subset, sel_concepts, sel_length, sel_score = set(), set(), 0, 0

        # greedily select a sentence
        while True:

            ###################################################################
            # RETRIEVE THE BEST SENTENCE
            ###################################################################

            # sort the sentences by gain and reverse length
            sort_sent = sorted(((weights[i] / float(self.sentences[i].length),
                                 -self.sentences[i].length,
                                 i)
                                for i in sentences),
                               reverse=True)

            # select the first sentence that fits in the length limit
            for sentence_gain, rev_length, sentence_index in sort_sent:
                if sel_length - rev_length <= summary_size:
                    break
            # if we don't find a sentence, break out of the main while loop
            else:
                break

            # if the gain is null, break out of the main while loop
            if not weights[sentence_index]:
                break

            # update the selected subset properties
            sel_subset.add(sentence_index)
            sel_score += weights[sentence_index]
            sel_length -= rev_length

            # update sentence weights with the reverse index
            for concept in set(self.sentences[sentence_index].concepts):
                if concept not in sel_concepts:
                    for sentence in self.c2s[concept]:
                        weights[sentence] -= self.weights[concept]

            # update the last selected subset property
            sel_concepts.update(self.sentences[sentence_index].concepts)

        # check if a singleton has a better score than our greedy solution
        if best_singleton_score > sel_score:
            return best_singleton_score, set([best_singleton])

        # returns the (objective function value, solution) tuple
        return sel_score, sel_subset

    def tabu_search(self,
                    summary_size=100,
                    memory_size=10,
                    iterations=100,
                    mutation_size=2,
                    mutation_group=True):
        """Greedy approximation of the ILP model with a tabu search
          meta-heuristic.
        Args:
            summary_size (int): the maximum size in words of the summary,
              defaults to 100.
            memory_size (int): the maximum size of the pool of sentences
              to ban at a given time, defaults at 5.
            iterations (int): the number of iterations to run, defaults at
              30.
            mutation_size (int): number of sentences to unselect and add to
              the tabu list at each iteration.
            mutation_group (boolean): flag to consider the mutations as a
              group: we'll check sentence combinations in the tabu list, not
              sentences alone.
        Returns:
            (value, set) tuple (int, list): the value of the approximated
              objective function and the set of selected sentences as a tuple.
        """
        # compute concept to sentences and concept sets for each sentence
        if not self.c2s:
            self.compute_c2s()
        if not self.concept_sets:
            self.compute_concept_sets()

        # initialize weights
        weights = {}

        # initialize the score of the best singleton
        best_singleton_score = 0

        # compute initial weights and fill the reverse index
        # while keeping track of the best singleton solution
        for i, sentence in enumerate(self.sentences):
            weights[i] = sum(self.weights[c] for c in set(sentence.concepts))
            if sentence.length <= summary_size\
               and weights[i] > best_singleton_score:
                best_singleton_score = weights[i]
                best_singleton = i

        best_subset, best_score = None, 0
        state = State()
        for i in xrange(iterations):
            queue = deque([], memory_size)
            # greedily select sentences
            state = self.select_sentences(summary_size,
                                          weights,
                                          state,
                                          queue,
                                          mutation_group)
            if state.score > best_score:
                best_subset = state.subset.copy()
                best_score = state.score
            to_tabu = set(random.sample(state.subset, mutation_size))
            state = self.unselect_sentences(weights, state, to_tabu)
            queue.extend(to_tabu)

        # check if a singleton has a better score than our greedy solution
        if best_singleton_score > best_score:
            return best_singleton_score, set([best_singleton])

        # returns the (objective function value, solution) tuple
        return best_score, best_subset

    def select_sentences(self,
                         summary_size,
                         weights,
                         state,
                         tabu_set,
                         mutation_group):
        """Greedy sentence selector.
        Args:
            summary_size (int): the maximum size in words of the summary,
              defaults to 100.
            weights (dictionary): the sentence weights dictionary. This
              dictionnary is updated during this method call (in-place).
            state (State): the state of the tabu search from which to start
              selecting sentences.
            tabu_set (iterable): set of sentences that are tabu: this
              selector will not consider them.
            mutation_group (boolean): flag to consider the mutations as a
              group: we'll check sentence combinations in the tabu list, not
              sentences alone.
        Returns:
            state (State): the new state of the search. Also note that
              weights is modified in-place.
        """
        # greedily select a sentence while respecting the tabu
        while True:

            ###################################################################
            # RETRIEVE THE BEST SENTENCE
            ###################################################################

            # sort the sentences by gain and reverse length
            sort_sent = sorted(((weights[i] / float(self.sentences[i].length),
                                 -self.sentences[i].length,
                                 i)
                                for i in range(len(self.sentences))
                                if self.sentences[i].length + state.length <=
                                summary_size),
                               reverse=True)

            # select the first sentence that fits in the length limit
            for sentence_gain, rev_length, sentence_index in sort_sent:
                if mutation_group:
                    subset = state.subset | {sentence_index}
                    for tabu in tabu_set:
                        if tabu <= subset:
                            break
                    else:
                        break
                else:
                    if sentence_index not in tabu_set:
                        break
            # if we don't find a sentence, break out of the main while loop
            else:
                break

            # if the gain is null, break out of the main while loop
            if not weights[sentence_index]:
                break

            # update state
            state.subset |= {sentence_index}
            state.concepts.update(self.concept_sets[sentence_index])
            state.length -= rev_length
            state.score += weights[sentence_index]

            # update sentence weights with the reverse index
            for concept in set(self.concept_sets[sentence_index]):
                if state.concepts[concept] == 1:
                    for sentence in self.c2s[concept]:
                        weights[sentence] -= self.weights[concept]
        return state

    def unselect_sentences(self, weights, state, to_remove):
        """Sentence ``un-selector'' (reverse operation of the
          select_sentences method).
        Args:
            weights (dictionary): the sentence weights dictionary. This
              dictionnary is updated during this method call (in-place).
            state (State): the state of the tabu search from which to start
              un-selecting sentences.
            to_remove (iterable): set of sentences to unselect.
        Returns:
            state (State): the new state of the search. Also note that
              weights is modified in-place.
        """
        # remove the sentence indices from the solution subset
        state.subset -= to_remove
        for sentence_index in to_remove:
            # update state
            state.concepts.subtract(self.concept_sets[sentence_index])
            state.length -= self.sentences[sentence_index].length
            # update sentence weights with the reverse index
            for concept in set(self.concept_sets[sentence_index]):
                if not state.concepts[concept]:
                    for sentence in self.c2s[concept]:
                        weights[sentence] += self.weights[concept]
            state.score -= weights[sentence_index]
        return state

    def solve_ilp_problem(self,
                          summary_size=100,
                          solver='glpk',
                          excluded_solutions=[],
                          unique=False):
        """Solve the ILP formulation of the concept-based model.
        Args:
            summary_size (int): the maximum size in words of the summary,
              defaults to 100.
            solver (str): the solver used, defaults to glpk.
            excluded_solutions (list of list): a list of subsets of sentences
              that are to be excluded, defaults to []
            unique (bool): modify the model so that it produces only one optimal
              solution, defaults to False
        Returns:
            (value, set) tuple (int, list): the value of the objective function
              and the set of selected sentences as a tuple.
        """
        # initialize container shortcuts
        concepts = self.weights.keys()
        w = self.weights
        L = summary_size
        C = len(concepts)
        S = len(self.sentences)

        if not self.word_frequencies:
            self.compute_word_frequency()

        tokens = self.word_frequencies.keys()
        f = self.word_frequencies
        T = len(tokens)

        # HACK Sort keys
        concepts = sorted(self.weights, key=self.weights.get, reverse=True)

        # formulation of the ILP problem
        prob = pulp.LpProblem(self.input_directory, pulp.LpMaximize)

        # initialize the concepts binary variables
        c = pulp.LpVariable.dicts(name='c_var',
                                  indexs=range(C),
                                  lowBound=0,
                                  upBound=1,
                                  cat='Integer')

        # initialize the sentences binary variables
        s = pulp.LpVariable.dicts(name='s_var',
                                  indexs=range(S),
                                  lowBound=0,
                                  upBound=1,
                                  cat='Integer')

        # initialize the word binary variables
        t = pulp.LpVariable.dicts(name='t_var',
                                  indexs=range(T),
                                  lowBound=0,
                                  upBound=1,
                                  cat='Integer')

        # OBJECTIVE FUNCTION
        prob += sum(w[concepts[i]] * c[i] for i in range(C))
               
        if unique:
            prob += sum(w[concepts[i]] * c[i] for i in range(C)) + \
                    10e-6 * sum(f[tokens[k]] * t[k] for k in range(T))

        # CONSTRAINT FOR SUMMARY SIZE
        prob += sum(s[j] * self.sentences[j].length for j in range(S)) <= L

        # INTEGRITY CONSTRAINTS
        for i in range(C):
            for j in range(S):
                if concepts[i] in self.sentences[j].concepts:
                    prob += s[j] <= c[i]

        for i in range(C):
            prob += sum(s[j] for j in range(S)
                        if concepts[i] in self.sentences[j].concepts) >= c[i]

        # WORD INTEGRITY CONSTRAINTS
        if unique:
            for k in range(T):
                for j in self.w2s[tokens[k]]:
                    prob += s[j] <= t[k]

            for k in range(T):
                prob += sum(s[j] for j in self.w2s[tokens[k]]) >= t[k]

        # CONSTRAINTS FOR FINDING OPTIMAL SOLUTIONS
        for sentence_set in excluded_solutions:
            prob += sum([s[j] for j in sentence_set]) <= len(sentence_set)-1

        # prob.writeLP('test.lp')

        # solving the ilp problem
        if solver == 'gurobi':
            prob.solve(pulp.GUROBI(msg=0),)
        elif solver == 'glpk':
            prob.solve(pulp.PULP_CBC_CMD(msg=0),)
        elif solver == 'cplex':
            prob.solve(pulp.CPLEX(msg=0),)
        else:
            sys.exit('no solver specified')

        # retreive the optimal subset of sentences
        solution = set([j for j in range(S) if s[j].varValue == 1])

        # returns the (objective function value, solution) tuple
        return (pulp.value(prob.objective), solution)


## Summary with ILP model

In [ ]:

def summary_text(dir_path):
    """
    Summary text document. 
    Args: 
        dir_path: directory to the document. The document is expected to has one sentence each line.
    """
    s = ConceptBasedILPSummarizer(dir_path)
    s.read_documents(file_extension="txt")
    s.extract_ngrams()
    s.compute_document_frequency()
    s.prune_sentences(mininum_sentence_length=10,
                    remove_citations=True,
                    remove_redundancy=True)
    value, subset = s.solve_ilp_problem()

    summary = '\n'.join([s.sentences[j].untokenized_form for j in subset])
    return summary

### Util function help split text to one sentence each line

In [ ]:
# sentence_tokenizer  =  nltk.data.load('tokenizers/punkt/english.pickle')
def split_sentence(data_path):
    filenames = os.listdir(data_path)

    for filename in filenames:
        lines = []
        with open(os.path.join(data_path, filename), 'r',errors = 'replace' ) as f:
            line = f.read()
            lines.extend(sentence_tokenizer.tokenize(line.rstrip()))
     
        with open(os.path.join(data_path, filename), 'w',errors = 'replace') as f:
            for line in lines:
                f.write(line + '\n')
    # for filename in filenames:
    #     print(filename)
    #     with open(os.path.join(data_path, filename), 'w') as f:
    #         for line in lines:
    #             f.write(line + '\n')




# path = '/content/drive/MyDrive/nlp projects/text_summarization/BBC News Summary/News Articles/tech'
# split_sentence(path)

In [ ]:
summary_text('/content/drive/MyDrive/nlp projects/text_summarization/BBC News Summary/News Articles/business/009.txt')

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


'In 2003, crop production totalled 11.49 million tonnes, the joint report from the Food and Agriculture Organisation and the World Food Programme said.\nGood rains, increased use of fertilizers and improved seeds contributed to the rise in production.\nIn eastern and southern Ethiopia, a prolonged drought has killed crops and drained wells.\n"Local purchase of cereals for food assistance programmes is recommended as far as possible, so as to assist domestic markets and farmers," said Henri Josserand, chief of FAO\'s Global Information and Early Warning System.\nAgriculture is the main economic activity in Ethiopia, representing 45% of gross domestic product.'

### Rouge scoring

In [5]:

scorer = rouge_scorer.RougeScorer([ 'rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def rouge_scoring(data_path, sum_path):
    all_summaries = []
    filenames = os.listdir(data_path)
    for filename in filenames:
        all_summaries.append(summary_text(os.path.join(data_path, filename)))

    for i  in range(len(all_summaries)):
        all_summaries[i] = all_summaries[i].replace('\n', ' ')

    scores = []
    
    for i  in range(len(filenames)):
        with open(os.path.join(sum_path, filenames[i]), 'r') as f:
            reference = f.read().rstrip()
            scores.append(scorer.score(reference, all_summaries[i] ))


    all_rouge1 = []
    all_rouge2 = []
    all_rougeL = []
    for dic in scores:
        all_rouge1.append(dic['rouge1'][2])
        all_rouge2.append(dic['rouge2'][2])
        all_rougeL.append(dic['rougeL'][2])

    return (np.mean(all_rouge1), np.mean(all_rouge2), np.mean(all_rougeL))




    

In [ ]:
# summary and scoring all the document in a dir. 
data_path = '/content/drive/MyDrive/nlp projects/text_summarization/BBC News Summary/News Articles/business'
sum_path = '/content/drive/MyDrive/nlp projects/text_summarization/BBC News Summary/Summaries/business'

scores = rouge_scoring(data_path, sum_path)

In [ ]:
# Available solver
pulp.list_solvers(onlyAvailable= True)

['PULP_CBC_CMD', 'PULP_CHOCO_CMD']

## Sume CNN dataset

In [ ]:
import pandas as pd
cnn = pd.read_csv('/content/cnn.csv',error_bad_lines=False, index_col = 0)

In [ ]:
for i in range(len(cnn)):
    cnn.iloc[i, 0] = re.sub(r'b"', '', cnn.iloc[i, 0])
    cnn.iloc[i, 1] = re.sub(r'b"', '', cnn.iloc[i, 1])


In [ ]:
# sentence_tokenizer  =  nltk.data.load('tokenizers/punkt/english.pickle')

# cnn_path = '/content/drive/MyDrive/nlp projects/text_summarization/CNN_2000'
# for i in range(len(cnn)):
#     all_lines = sentence_tokenizer.tokenize(cnn.iloc[i, 0])
#     with open(os.path.join(cnn_path, 'article', str(i) + '.txt'), 'w') as f:
#         for line in all_lines:
#             f.write(line + '\n')
#     with open(os.path.join(cnn_path, 'summary', str(i) + '.txt'), 'w') as f:
#         f.write(cnn.iloc[i, 1])




In [ ]:
data_path = '/content/drive/MyDrive/nlp projects/text_summarization/CNN_2000/article'
sum_path = '/content/drive/MyDrive/nlp projects/text_summarization/CNN_2000/summary'

scores = rouge_scoring(data_path, sum_path)

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
scores

(0.2919275486473456, 0.09819238611267934, 0.1755400098591794)

# BERT-extractive-summarizer

In [6]:
!pip install bert-extractive-summarizer

     |████████████████████████████████| 1.5MB 16.3MB/s 
     |████████████████████████████████| 2.9MB 51.8MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7cb758e967e8f295b4452049d40f0338ea43d2f2c7bf9e4338962e70eee7a5cd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [18]:
from summarizer import Summarizer
model = Summarizer()

In [ ]:
s = open('/content/drive/MyDrive/nlp projects/text_summarization/BBC News Summary/News Articles/business/009.txt', 'r').read()
model(s, ratio = 0.35)

"Ethiopia's crop production up 24%\n\nEthiopia produced 14.27 million tonnes of crops in 2004, 24% higher than in 2003 and 21% more than the average of the past five years, a report says. Nevertheless, 2.2 million Ethiopians will still need emergency assistance. Last year, a total of 965,000 tonnes of food assistance was needed to help seven million Ethiopians. The Food and Agriculture Organisation (FAO) recommend that the food assistance is bought locally. Agriculture is the main economic activity in Ethiopia, representing 45% of gross domestic product."

In [19]:


def rouge_score_bert(df):
    all_scores = []
    for i in range(len(df)):
        candidate = model(df.iloc[i, 0], ratio = 0.5)
        all_scores.append(scorer.score(df.iloc[i, 1] ,candidate),)
        
    return all_scores



In [29]:
import pandas as pd
df = pd.read_csv('/content/BBC.csv',)
df.head()

,article,summary
0,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...
1,Ad sales boost Time Warner profit\nQuarterly p...,TimeWarner said fourth quarter sales rose 2% t...
2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...
3,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...
4,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ..."


In [14]:
import re
for i in range(len(df)):
    df.iloc[i, 0] = re.sub(r"b'", '', df.iloc[i, 0])
    df.iloc[i, 0] = re.sub(r'b"', '', df.iloc[i, 0])

    df.iloc[i, 1] = re.sub(r"b'", '', df.iloc[i, 1])
    df.iloc[i, 1] = re.sub(r'b"', '', df.iloc[i, 1])


In [30]:
all_scores = rouge_score_bert(df)

In [31]:
all_scores[:5]

[{'rouge1': Score(precision=0.6440677966101694, recall=0.6705882352941176, fmeasure=0.6570605187319885),
  'rouge2': Score(precision=0.5, recall=0.5207100591715976, fmeasure=0.5101449275362319),
  'rougeL': Score(precision=0.4180790960451977, recall=0.43529411764705883, fmeasure=0.42651296829971186)},
 {'rouge1': Score(precision=0.49783549783549785, recall=0.7516339869281046, fmeasure=0.5989583333333334),
  'rouge2': Score(precision=0.3695652173913043, recall=0.5592105263157895, fmeasure=0.4450261780104712),
  'rougeL': Score(precision=0.26406926406926406, recall=0.39869281045751637, fmeasure=0.31770833333333337)},
 {'rouge1': Score(precision=0.6038961038961039, recall=0.7153846153846154, fmeasure=0.6549295774647886),
  'rouge2': Score(precision=0.477124183006536, recall=0.5658914728682171, fmeasure=0.5177304964539008),
  'rougeL': Score(precision=0.3961038961038961, recall=0.46923076923076923, fmeasure=0.4295774647887323)},
 {'rouge1': Score(precision=0.5460526315789473, recall=0.7094

In [32]:
all_r1 = []
all_r2 = []
all_l = []
for score in all_scores:
    all_r1.append(score['rouge1'][2])
    all_r2.append(score['rouge2'][2])
    all_l.append(score['rougeL'][2])

In [35]:
np.mean(all_l
        )

0.3881354250227339

In [ ]:
all_scores[0]

{'rouge1': Score(precision=0.1559633027522936, recall=0.4857142857142857, fmeasure=0.23611111111111113),
 'rouge2': Score(precision=0.046296296296296294, recall=0.14705882352941177, fmeasure=0.0704225352112676),
 'rougeL': Score(precision=0.11009174311926606, recall=0.34285714285714286, fmeasure=0.16666666666666669)}

In [ ]:
def bert_bbc(data_path, sum_path):
    filenames = os.listdir(data_path)
    for filename in filenames:
        article = open(os.path.join)

{'rouge1': Score(precision=0.21052631578947367, recall=0.34285714285714286, fmeasure=0.2608695652173913),
 'rouge2': Score(precision=0.07142857142857142, recall=0.11764705882352941, fmeasure=0.08888888888888889),
 'rougeL': Score(precision=0.12280701754385964, recall=0.2, fmeasure=0.15217391304347827)}

In [ ]:
def bert_summary_text(file_path):
    text = ''
    with open(file_path, 'r') as f:
        for line in f.readlines():
            text += line.rstrip()
    return text
